# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Import Clean City Data
weather = os.path.join('..', 'WheaterPy','Output_Data','Clean_City_Data.csv')
weather_df = pd.read_csv(weather)
weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Clyde River,70.4692,-68.5914,-0.87,86,40,7.72,CA,1622047223
1,Lebu,-37.6167,-73.6500,15.92,81,98,3.63,CL,1622047165
2,Busselton,-33.6500,115.3333,11.69,64,100,5.46,AU,1622047242
3,Rikitea,-23.1203,-134.9692,24.84,79,38,11.23,PF,1622047221
4,Alofi,-19.0595,-169.9187,23.94,94,88,1.54,NU,1622047344
...,...,...,...,...,...,...,...,...,...
562,Jaciara,-15.9653,-54.9683,33.28,39,6,0.73,BR,1622047966
563,Verkhnevilyuysk,63.4453,120.3167,6.32,88,91,1.69,RU,1622047966
564,Lahaina,20.8783,-156.6825,23.03,83,36,3.10,US,1622047967
565,Superior,46.7208,-92.1041,10.67,55,75,7.72,US,1622047967


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Store latitude and longitude in locations
world_location = weather_df[['Lat', 'Lng']]

# Store humidity
humidity = weather_df['Humidity (%)']

In [27]:
# Plot Heatmap
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(world_location, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=0.3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
fitting_weather = weather_df.loc[(weather_df['Max Temp (C)']>=18) & (weather_df['Max Temp (C)']<=33) & (weather_df['Wind Speed (mph)']<=18) & (weather_df['Cloudiness (%)'] == 0),:]
fitting_weather.reset_index(drop=True)

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Terenos,-20.4422,-54.8603,30.84,45,0,2.53,BR,1622047832
1,Miran Shah,33.0086,70.0697,29.48,30,0,1.18,PK,1622047833
2,Abhā,18.2164,42.5053,23.14,60,0,1.97,SA,1622047373
3,Santa Fe,-31.6333,-60.7000,22.22,41,0,1.79,AR,1622047574
4,Abu Samrah,35.3029,37.1841,28.43,22,0,5.44,SY,1622047849
5,Bykovo,55.6361,38.0803,21.63,43,0,3.63,RU,1622047854
6,Cidreira,-30.1811,-50.2056,19.08,48,0,2.69,BR,1622047859
7,Valparaíso,-33.0393,-71.6273,23.34,29,0,5.14,CL,1622047647
8,Talara,-4.5772,-81.2719,25.42,65,0,6.69,PE,1622047866
9,Mahon,39.8885,4.2658,19.31,55,0,5.14,ES,1622047876


In [36]:
fitting_weather_df = fitting_weather.drop(fitting_weather.index[24])
fitting_weather_df.reset_index(drop=True)

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Terenos,-20.4422,-54.8603,30.84,45,0,2.53,BR,1622047832
1,Miran Shah,33.0086,70.0697,29.48,30,0,1.18,PK,1622047833
2,Abhā,18.2164,42.5053,23.14,60,0,1.97,SA,1622047373
3,Santa Fe,-31.6333,-60.7000,22.22,41,0,1.79,AR,1622047574
4,Abu Samrah,35.3029,37.1841,28.43,22,0,5.44,SY,1622047849
5,Bykovo,55.6361,38.0803,21.63,43,0,3.63,RU,1622047854
6,Cidreira,-30.1811,-50.2056,19.08,48,0,2.69,BR,1622047859
7,Valparaíso,-33.0393,-71.6273,23.34,29,0,5.14,CL,1622047647
8,Talara,-4.5772,-81.2719,25.42,65,0,6.69,PE,1622047866
9,Mahon,39.8885,4.2658,19.31,55,0,5.14,ES,1622047876


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [54]:
hotel_df = []
# target_search = 'hotel'
target_radius = 5000
target_type = "lodging"

for city in range(len(fitting_weather_df['City'])):
    lat = fitting_weather_df.iloc[city]['Lat']
    lng = fitting_weather_df.iloc[city]['Lng']
    coords = f'{lat},{lng}'
    params = {
    "location": coords,
#     "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key}
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    try:
        hotel_df.append(response["results"][0]["name"])    
    except:
        hotel_df.append('No lodging nearby')

In [55]:
fitting_weather_df['Hotel Name'] = hotel_df
fitting_weather_df

,City,Lat,Lng,Max Temp (C),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
57,Terenos,-20.4422,-54.8603,30.84,45,0,2.53,BR,1622047832,Rancho Pitzer
62,Miran Shah,33.0086,70.0697,29.48,30,0,1.18,PK,1622047833,Bachelor Hostel
86,Abhā,18.2164,42.5053,23.14,60,0,1.97,SA,1622047373,Abha Hotel
105,Santa Fe,-31.6333,-60.7000,22.22,41,0,1.79,AR,1622047574,Conquistador Hotel
140,Abu Samrah,35.3029,37.1841,28.43,22,0,5.44,SY,1622047849,No lodging nearby
159,Bykovo,55.6361,38.0803,21.63,43,0,3.63,RU,1622047854,Around the world
181,Cidreira,-30.1811,-50.2056,19.08,48,0,2.69,BR,1622047859,Hotel Castelo
197,Valparaíso,-33.0393,-71.6273,23.34,29,0,5.14,CL,1622047647,Hotel Ultramar
207,Talara,-4.5772,-81.2719,25.42,65,0,6.69,PE,1622047866,Hostal Oro Negro
240,Mahon,39.8885,4.2658,19.31,55,0,5.14,ES,1622047876,ARTIEM Capri


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [53]:
# Add marker layer ontop of heat map
# Assign the marker layer to a variable
markers = gmaps.marker_layer(locations)
# Add the layer to the map
fig.add_layer(markers)


# Display figure
fig

NameError: name 'locations' is not defined